# Si decimos que un usuario sabe un idioma cuando tiene un nivel de babel mayor o igual a 1, para cada grupo de usuarios que sabe una determinada cantidad de idiomas, ¿Cuántos de esos usuarios fueron bloqueados al menos una vez? (⭐⭐⭐)

In [1]:
import pandas as pd
lang = pd.read_csv('../data/languages.csv')
lang

,babel_user,babel_lang,babel_level
0,4502458,FR,2
1,5928200,aa,0
2,46918,ab,0
3,2050449,ab,1
4,4715583,ace,0
...,...,...,...
35661,2089885,zh-Hant-HK,1
35662,2089885,zh-Hant-TW,3
35663,5036533,zu,0
35664,5928200,zu,0


In [2]:
lang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35666 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   babel_user   35666 non-null  int64 
 1   babel_lang   35662 non-null  object
 2   babel_level  35666 non-null  object
dtypes: int64(1), object(2)
memory usage: 836.1+ KB


In [3]:
lang.babel_level.unique()

array(['2', '0', '1', '3', '4', 'N', '5'], dtype=object)

In [4]:
# Cambio el dtype de babeL_level a enteros.
# Asigno a los nativos a nivel 5 y los xx-5 a nivel 6.
def traducir_nivel(string):
    traduccion = {
        '0':0,
        '1':1,
        '2':2,
        '3':3,
        '4':4,
        'N':5,
        '5':6,
    }

    return traduccion[string]

lang.babel_level = lang.babel_level.map(traducir_nivel)
lang.babel_user = lang.babel_user.map(lambda x: str(x))
# Considero las provenencias de los idiomas como el mismo idioma.
lang.babel_lang = lang.babel_lang.map(lambda x: str(x).split('-')[0].lower())
lang

,babel_user,babel_lang,babel_level
0,4502458,fr,2
1,5928200,aa,0
2,46918,ab,0
3,2050449,ab,1
4,4715583,ace,0
...,...,...,...
35661,2089885,zh,1
35662,2089885,zh,3
35663,5036533,zu,0
35664,5928200,zu,0


In [5]:
# Calculo la cantidad de idiomas que habla cada usuario.
lang = lang[lang.babel_level >= 1].groupby('babel_user').count()
lang = lang.reset_index().rename(columns={'babel_user':'users', 'babel_lang':'nlangs'})[['users', 'nlangs']]
lang

,users,nlangs
0,1000475,3
1,1001148,2
2,100131,5
3,1001364,7
4,1001897,2
...,...,...
10804,998096,1
10805,998130,2
10806,998232,2
10807,99851,3


In [6]:
# Consigo una set de usuarios bloqueados.
logs = pd.read_csv('../data/logs.csv', low_memory=False, usecols=['action','title'])
logs = logs[logs.action == 'block'].drop('action', axis=1)
bloqueados = set(logs.title.map(lambda x: str(x).replace('Usuario:','')))
bloqueados

{'186.176.29.125',
 '190.67.163.214',
 '201.240.48.150',
 'Ortjomon',
 '200.86.75.27',
 '189.158.121.183',
 'LifeHax111',
 '186.190.196.199',
 '190.107.228.66',
 '189.128.99.51',
 'Dob Remonno',
 '84.79.134.62',
 'Addsmooaw',
 '147.156.173.214',
 'Bosterodeboedo',
 '179.50.159.71',
 '217.216.64.143',
 '181.203.4.123',
 'Raiza8888',
 'Vito Bandoneone',
 '83.213.87.199',
 '190.226.222.0/24',
 '201.223.101.192',
 '200.121.34.99',
 '200.114.40.96',
 '189.226.218.241',
 '128.232.18.57',
 'SrAlejandro29',
 '201.167.115.248',
 'Pasco+Moquegua',
 '152.231.37.152',
 'Agustin diaz gonzalez',
 'Danirecu26',
 '158.251.7.68',
 '88.13.155.173',
 '81.32.192.173',
 '83.35.171.185',
 '201.116.199.235',
 '200.89.56.210',
 '181.133.228.34',
 'Eldestrozacul0s',
 '189.141.27.162',
 '190.76.83.120',
 '186.129.198.146',
 '88.7.242.112',
 '187.230.60.134',
 '001010010010 alejandro',
 '190.42.53.201',
 '189.165.113.104',
 '190.224.252.0/24',
 'Milanopablog66',
 '181.47.148.53',
 '85.85.95.34',
 '201.240.161.12

In [7]:
# Como bloqueados tiene mezclados los username y las ips y el df langs tiene sus ids, tengo pasar a ids los bloqueados.
users = pd.read_csv('../data/logs.csv', low_memory=False, usecols=['contributor_username','contributor_id','contributor_ip'])
users

,contributor_username,contributor_id,contributor_ip
0,Sabbut,2075.0,NaN
1,Sabbut,2075.0,NaN
2,Sabbut,2075.0,NaN
3,Sanbec,7856.0,NaN
4,Sanbec,7856.0,NaN
...,...,...,...
12188941,SeroBOT,4980693.0,NaN
12188942,Leonpolanco,561850.0,NaN
12188943,Desiretjs,6335019.0,NaN
12188944,Correogsk,443260.0,NaN


In [8]:
# Paso contributor_id a string y me quedo con la parte entera.
users.contributor_id = users.contributor_id.map(lambda x: str(x).split('.')[0])
# Lleno los NaN con '' para poder concatenar.
users.fillna('', inplace=True)
users

,contributor_username,contributor_id,contributor_ip
0,Sabbut,2075,
1,Sabbut,2075,
2,Sabbut,2075,
3,Sanbec,7856,
4,Sanbec,7856,
...,...,...,...
12188941,SeroBOT,4980693,
12188942,Leonpolanco,561850,
12188943,Desiretjs,6335019,
12188944,Correogsk,443260,


In [9]:
# Agrupo el DF por ids y me quedo con el nombre e ip del usuario.
users = users.groupby('contributor_id').agg({'contributor_username':'first', 'contributor_ip':'first'})
users

,contributor_username,contributor_ip
contributor_id,,
100000,Sophymarsho,
1000000,Antimarcha,
1000001,Molusquita,
1000002,Heidy Carvajal,
1000003,Vierko,
...,...,...
999996,Cristian carrillo,
999997,Adalyth,
999998,Sir.rizas,


In [10]:
# Creo una columna con el id (username si no es nan o ip si era nan) de cada usuario.
users['nombre_ip'] = users.contributor_username + users.contributor_ip
users.drop(['contributor_username', 'contributor_ip'], axis=1, inplace=True)
users = users.reset_index()
users

,contributor_id,nombre_ip
0,100000,Sophymarsho
1,1000000,Antimarcha
2,1000001,Molusquita
3,1000002,Heidy Carvajal
4,1000003,Vierko
...,...,...
6188934,999996,Cristian carrillo
6188935,999997,Adalyth
6188936,999998,Sir.rizas
6188937,999999,Ronaldrivera


In [11]:
# Paso bloqueados a ids.
bloqueados = users[users.nombre_ip.map(lambda x: x in bloqueados)].contributor_id
bloqueados

50         1000046
65          100006
172        1000157
391        1000360
485        1000447
            ...   
6188449     999549
6188516     999611
6188571     999661
6188628     999714
6188843     999912
Name: contributor_id, Length: 95448, dtype: object

In [12]:
lang

,users,nlangs
0,1000475,3
1,1001148,2
2,100131,5
3,1001364,7
4,1001897,2
...,...,...
10804,998096,1
10805,998130,2
10806,998232,2
10807,99851,3


In [18]:
# Consigo un DF de los usuarios bloqueados.
users_bloqueados = pd.merge(users, bloqueados, on='contributor_id')

# Mergeo con lang para conseguir la cantidad de idiomas que habla cada usuario.
lang = lang.rename(columns={'users':'contributor_id'})
users_bloqueados = pd.merge(users_bloqueados, lang, on='contributor_id')
users_bloqueados

,contributor_id,nombre_ip,nlangs
0,1001897,Oswaldo va,2
1,1013041,Surfaz,2
2,1025077,Sfs90,3
3,102876,Keat,22
4,1029705,Hiddendaemian,3
...,...,...,...
605,983790,Orgus,1
606,988086,The big.1,2
607,989000,ADHUC TEMPUS,6
608,99159,RoRo,3


In [19]:
# Agrupo por cantidad de idiomas y consigo el dato de si hay algun bloqueado por grupo.
users_bloqueados.groupby('nlangs').agg({'contributor_id':'count'}).rename(columns={'contributor_id':'cantidad_bloqueados'})
# Estos son los grupos de usuarios (por cantidad de idiomas que hablan) que tienen por lo menos un bloqueado.

,cantidad_bloqueados
nlangs,
1,93
2,182
3,138
4,82
5,52
6,34
7,10
8,8
10,4
